SELL FRACTION ALSO CALCULATED AS KELLY

In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ta
import vnstock as vn

# Parameters
RSI_PERIOD = 14
RSI_OVERSOLD = 30
RSI_OVERBOUGHT = 70
MACD_SLOW_PERIOD = 26
MACD_FAST_PERIOD = 12
MACD_SIGNAL_PERIOD = 9
initial_investment = 120000000
backup_amount_initial = 40_000_000

In [35]:
win_rate = 0.5521109556
loss_rate = 1 - win_rate
mean_profit = 0.3944941585
mean_loss = 0.1314483279

In [36]:
def kelly_criterion(p, q, profit, loss):
    return (p/loss) - (q/profit)

In [37]:
kelly_criterion(win_rate,loss_rate,mean_profit,mean_loss)

3.064862589876477

In [38]:

companies_vn30 = [
    'SSI', 'BCM','VHM','VIC','VRE','BVH','POW','GAS','ACB','BID',
'CTG','HDB','MBB','SSB','SHB','STB','TCB','TPB','VCB','VIB','VPB','HPG',
'GVR','MSN','VNM','SAB','VJC','MWG','PLX','FPT']

In [39]:
companies = ['SSI', 'SHB', 'STB']

In [40]:
def calculate_indicators(df):
    if df.empty:
        return df
    
    df['RSI'] = ta.momentum.RSIIndicator(df['close'], RSI_PERIOD).rsi()
    df['Previous_RSI'] = df['RSI'].shift(1)
    df['Previous_RSI'].fillna(0, inplace=True)
    macd = ta.trend.MACD(df['close'], window_slow=MACD_SLOW_PERIOD, window_fast=MACD_FAST_PERIOD, window_sign=MACD_SIGNAL_PERIOD)
    df['MACD'] = macd.macd()
    df['Signal_Line'] = macd.macd_signal()
    df['Previous_MACD'] = df['MACD'].shift(1)
    df['Previous_Signal_Line'] = df['Signal_Line'].shift(1)
    df['Previous_MACD'].fillna(0, inplace=True)
    df['Previous_Signal_Line'].fillna(0, inplace=True)

    return df

In [41]:
def macd_strategy(df):
    if df.empty:
        return df
    
    df['Signal'] = 0

    # Buy signals: RSI cross above 30 and MACD cross above Signal line
    df.loc[
        (df['Previous_MACD'] < df['Previous_Signal_Line']) &
        (df['MACD'] >= df['Signal_Line']) &
        (df['RSI'] > RSI_OVERSOLD), 'Signal'] = 1

    # Sell Signals: 
    df.loc[
        (df['RSI'] < RSI_OVERBOUGHT) &
        (df['Previous_MACD'] > df['Previous_Signal_Line']) &
        (df['MACD'] <= df['Signal_Line']), 'Signal'] = -1

    return df

In [42]:
def simulate_investment(ticker, win_rate, loss_rate, mean_profit, mean_loss, sell_fraction):
    try:
        data = vn.stock_historical_data(ticker, '2021-01-01', '2024-01-01', resolution='1D', type='stock')
        data = data.set_index(pd.DatetimeIndex(data['time'].values))
        data = calculate_indicators(data)
        data = macd_strategy(data)

        buy_signals = data[data['Signal'] == 1].index
        sell_signals = data[data['Signal'] == -1].index

        cash = initial_investment
        holdings = 0
        backup_amount = backup_amount_initial
        portfolio_values = []
        overspend = 0

        f_star = 0.25 * kelly_criterion(win_rate, loss_rate, mean_profit ,mean_loss)

        for i in range(len(data)):
            if data.index[i] in buy_signals:
                allocation = cash * f_star
                if allocation > 0:
                    shares_to_buy = int(allocation // data['close'][i])
                    if shares_to_buy > 0:
                        total_cost = shares_to_buy * data['close'][i]
                        if cash >= total_cost:
                            cash -= total_cost
                        else:
                            cash_needed = total_cost - cash
                            if cash_needed <= backup_amount:
                                cash = 0
                                backup_amount -= cash_needed
                                overspend += cash_needed
                            else: 
                                shares_to_buy = int((cash + backup_amount) // data['close'][i])
                                cash_needed = shares_to_buy * data['close'][i] - cash
                                cash = 0
                                backup_amount -= cash_needed
                                overspend += cash_needed
                        holdings += shares_to_buy
                        

            if data.index[i] in sell_signals and holdings > 0:
                current_price = data['close'].iloc[i]
                shares_to_sell = int(holdings * sell_fraction)
                if shares_to_sell > 0:
                    if shares_to_sell > holdings:
                        shares_to_sell = holdings
                    revenue = shares_to_sell * current_price 
                    cash += revenue
                    holdings -= shares_to_sell
                    # print(f"Selling {shares_to_sell} shares at {current_price} on {data.index[i]}")
                    #Check profit and repay backup amount if possible 
                    if cash > overspend:
                        repayment_amount = overspend
                        cash -= repayment_amount
                        overspend = 0
                        backup_amount += repayment_amount

            current_value = cash + holdings * data['close'].iloc[i] + backup_amount
            portfolio_values.append(current_value)
            # print(f"Day {data.index[i]}: Cash: {cash}, Holdings: {holdings}, Current Value: {current_value}")

        data['Portfolio_Value'] = portfolio_values
        data['Accumulated_Profit'] = data['Portfolio_Value'] - (initial_investment+backup_amount)

        return data
    except Exception as e:
        print(f"Error occurred for {ticker}: {e}")
        return pd.DataFrame()

In [43]:
def backtest_multiple_companies(companies_vn30, win_rate, loss_rate, mean_profit, mean_loss, sell_fraction):
    results = []
    for company in companies_vn30:
        result = simulate_investment(company, win_rate, loss_rate, mean_profit, mean_loss, sell_fraction)
        if not result.empty:
            results.append({
                'Company': company,
                'Final Portfolio Value': result['Portfolio_Value'].iloc[-1],
                'Total Profit': result['Accumulated_Profit'].iloc[-1],
                'Rate of Return': result['Accumulated_Profit'].iloc[-1] / (initial_investment+backup_amount_initial)  * 100
            })
    return pd.DataFrame(results)

In [44]:
sell_fraction = 0.25 * kelly_criterion(win_rate, loss_rate, mean_profit, mean_loss)
results_df = backtest_multiple_companies(companies_vn30, win_rate, loss_rate, mean_profit, mean_loss, sell_fraction)

# Save results to CSV
results_df.to_csv('result/MACD_kelly.csv', index=False)

# Print results
print(results_df)

average_rate_of_return = results_df['Rate of Return'].mean()
print("Average Rate of Return for 30 companies:", average_rate_of_return)

C:\Users\Admin\AppData\Local\Temp\ipykernel_16952\245259429.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Previous_RSI'].fillna(0, inplace=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_16952\245259429.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, w

   Company  Final Portfolio Value  Total Profit  Rate of Return
0      SSI              280415402     120415402       75.259626
1      BCM              172587120      12587120        7.866950
2      VHM              137184530     -22815470      -14.259669
3      VIC              153064782      -6935218       -4.334511
4      VRE              147446000     -12554000       -7.846250
5      BVH              135947660     -24052340      -15.032713
6      POW              138001536     -21998464      -13.749040
7      GAS              200735290      40735290       25.459556
8      ACB              176695090      16695090       10.434431
9      BID              196088320      36088320       22.555200
10     CTG              168706420       8706420        5.441512
11     HDB              176321580      16321580       10.200988
12     MBB              198391000      38391000       23.994375
13     SSB              147609340     -12390660       -7.744162
14     SHB              232414283      7

C:\Users\Admin\AppData\Local\Temp\ipykernel_16952\245259429.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Previous_RSI'].fillna(0, inplace=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_16952\245259429.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, w